In [ ]:
import sys
!{sys.executable} -m pip install langchain


In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install pinecone[grpc] langchain-pinecone

In [ ]:
%pip install pypdf

In [ ]:
%pip install langchain_community langchain_experimental lanngchain_openai

In [12]:
import langchain
print(langchain.__version__)


0.3.27


In [13]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
# Extract Data From the PDF File
def load_pdf_file(book):
    loader=DirectoryLoader(book,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [14]:
extracted_data=load_pdf_file(book='book/')

In [ ]:
#extracted_data

In [15]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
text_chunks=text_split(extracted_data)
print("length of text chunks",len(text_chunks))

length of text chunks 24071


In [ ]:
#text_chunks

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings=download_hugging_face_embeddings()

C:\Users\DIGANTA\AppData\Local\Temp\ipykernel_5796\1085893552.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
%pip install python-dotenv


In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
#OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384, 
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


In [ ]:
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
#os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

nicher code gulo lagbe na,,openai use korle lagto

In [27]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [28]:
docsearch

In [29]:
retriever= docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [30]:
retrieved_docs=retriever.invoke("What is HIV?")

In [31]:
retrieved_docs

[Document(id='e8e11c5e-20ef-4cb4-b71f-2c102b39ef9a', metadata={'author': 'Maxine A. Papadakis, Stephen J. McPhee, Michael W. Rabow, Kenneth R. McQuaid', 'creationdate': '2021-08-16T16:32:47+00:00', 'creator': 'Adobe Distiller dc', 'moddate': '2022-03-09T21:12:40-03:00', 'page': 1392.0, 'page_label': '1393', 'producer': 'Adobe PDF Library 15.0', 'source': 'book\\Medical Book.pdf', 'title': 'CURRENT Medical Diagnosis and Treatment 2022', 'total_pages': 1874.0}, page_content='S\ue045ciety \ue045fAmerica. https://clinicalinf\ue045.hiv.g\ue045v/sites/default/files/guide-lines/d\ue045cuments/Adult_OI.pdf. Updated 2020 May 26.CMDT22_Ch31_p1331-p1373.indd'),
 Document(id='2bc769ad-178b-4fcc-bded-5b62dfff3639', metadata={'author': 'Maxine A. Papadakis, Stephen J. McPhee, Michael W. Rabow, Kenneth R. McQuaid', 'creationdate': '2021-08-16T16:32:47+00:00', 'creator': 'Adobe Distiller dc', 'moddate': '2022-03-09T21:12:40-03:00', 'page': 1406.0, 'page_label': '1407', 'producer': 'Adobe PDF Library 1

In [36]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4, max_tokens=500)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [39]:
question_answer_chain= create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)